In [25]:
import pandas as pd

In [26]:
import re

In [27]:
print(pd.__version__)

1.5.3


In [30]:
df = pd.read_csv('D:/Binar/Challenge/BinarChallenge1/AssetChallenge/abusive.csv')

In [31]:
df.head()

,ABUSIVE
0,alay
1,ampas
2,buta
3,keparat
4,anjing


In [32]:
df['ABUSIVE'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 125 entries, 0 to 124
Series name: ABUSIVE
Non-Null Count  Dtype 
--------------  ----- 
125 non-null    object
dtypes: object(1)
memory usage: 1.1+ KB


In [39]:
def censor(replaceword):
    abusiveword = df['ABUSIVE'].tolist()
    abusivearr = []
    for i, df in enumerate(abusiveword):
        abusivearr.append(df)
        for abusivearr in abusiveword:
            replaceword = re.sub(abusivearr, '*', * len(word), flags=re.IGNORECASE)
            print(f"Forbidden word REMOVED: {word}")
    return replaceword

In [40]:
print(censor)

<function censor at 0x000001E2FF6F85E0>


In [13]:
print(abusiveword)

0             alay
1            ampas
2             buta
3          keparat
4           anjing
          ...     
120          rezim
121          sange
122         serbet
123          sipit
124    transgender
Name: ABUSIVE, Length: 125, dtype: object


In [9]:
# def censor(replaceword):
#     for word in abusiveword:
#         pattern = r"\b{}\b".format(word)
#         if re.search(pattern, replaceword, re.IGNORECASE):
#             replaceword = re.sub(pattern, '*' * len(word), replaceword, flags=re.IGNORECASE)
#             print(f"Forbidden word REMOVED: {word}")
#     return replaceword

In [10]:
def censor(replaceword):
    for word in abusiveword:
        replaceword = re.sub(abusiveword, '*', * len(word), flags=re.IGNORECASE)
        print(f"Forbidden word REMOVED: {word}")
    return replaceword

In [11]:
print(censor)

<function censor at 0x000001E2F221B6A0>
